In [33]:
# fm training for anime

import xlearn as xl
import numpy as np
import pandas as pd

In [34]:
# 读取 csv
# dataset: https://www.kaggle.com/ajmichelutti/collaborative-filtering-on-anime-data/data?select=rating.csv
anime_train_df = pd.read_csv("data/anime.csv")
# 
# Get names of indexes for which column Stock has value No
indexNames = anime_train_df[ anime_train_df['episodes'] == 'Unknown' ].index
# print(indexNames)
# Delete these row indexes from dataFrame
anime_train_df.drop(indexNames , inplace=True)
anime_train_df = anime_train_df.reset_index()
anime_train_df.episodes.astype(int)
# print(anime_train_df.columns.tolist())
# print(anime_train_df.dtypes)
# drop 去除掉不需要的 column
anime_train_df = anime_train_df.drop(labels=['name', 'type', 'members', 'episodes'],axis=1)
anime_train_df

,index,anime_id,genre,rating
0,0,32281,"Drama, Romance, School, Supernatural",9.37
1,1,5114,"Action, Adventure, Drama, Fantasy, Magic, Mili...",9.26
2,2,28977,"Action, Comedy, Historical, Parody, Samurai, S...",9.25
3,3,9253,"Sci-Fi, Thriller",9.17
4,4,9969,"Action, Comedy, Historical, Parody, Samurai, S...",9.16
...,...,...,...,...
11949,12289,9316,Hentai,4.15
11950,12290,5543,Hentai,4.28
11951,12291,5621,Hentai,4.88
11952,12292,6133,Hentai,4.98


In [35]:
# 获取所有的 dataframe 每个 anime 的 genres 作为 list
df_genres_list = anime_train_df.genre.astype(str).tolist()


In [36]:

genre_type_map = {'Action': 0, 'Adventure': 1, 'Cars': 2, 'Comedy': 3, 'Dementia': 4,
'Demons': 5, 'Drama': 6, 'Ecchi': 7, 'Fantasy': 8, 'Game': 9, 'Harem': 10,
'Hentai': 11, 'Historical': 12, 'Horror': 13, 'Josei': 14, 'Kids': 15,
'Magic': 16, 'Martial Arts': 17, 'Mecha': 18, 'Military': 19, 'Music': 20,
'Mystery': 21, 'Parody': 22, 'Police': 23, 'Psychological': 24, 'Romance': 25,
'Samurai': 26, 'School': 27, 'Sci-Fi': 28, 'Seinen': 29, 'Shoujo': 30,
'Shoujo Ai': 31, 'Shounen': 32, 'Shounen Ai': 33, 'Slice of Life': 34,
'Space': 35, 'Sports': 36, 'Super Power': 37, 'Supernatural': 38,
'Thriller': 39, 'Vampire': 40, 'Yaoi': 41, 'Yuri': 42, 'nan': 43}
genre_type_list = ['Action', 'Adventure', 'Cars', 'Comedy', 'Dementia', 'Demons', 'Drama', 'Ecchi'
, 'Fantasy', 'Game', 'Harem', 'Hentai', 'Historical', 'Horror', 'Josei', 'Kids'
, 'Magic', 'Martial, Arts', 'Mecha', 'Military', 'Music', 'Mystery', 'Parody'
, 'Police', 'Psychological', 'Romance', 'Samurai', 'School', 'Sci-Fi', 'Seinen'
, 'Shoujo', 'Shoujo, Ai', 'Shounen', 'Shounen, Ai', 'Slice, of, Life', 'Space'
, 'Sports', 'Super, Power', 'Supernatural', 'Thriller', 'Vampire', 'Yaoi', 'Yuri'
, 'nan']

new_genres_list = []
arr_set = []
for i in range(len(df_genres_list)):
    # extract every anime's genres
    new_genres = np.zeros(len(genre_type_list) + 1)
    str_genres = str(df_genres_list[i]) 
    arr_splitted = str_genres.split(', ')
    for j in range(len(arr_splitted)): # one-hot encoding
        new_genres[genre_type_map[arr_splitted[j]]] = 1
    # add anime_id for after dataframe merge
    new_genres[len(genre_type_list)] = anime_train_df.anime_id[i] 
    new_genres = new_genres.astype(int).tolist()
    new_genres_list.append(new_genres)


In [37]:

# print(anime_train_df_new)
new_genres_array = np.array(new_genres_list)
anime_genres_columns = genre_type_list
anime_genres_columns.append('anime_id')
anime_train_genres_df = pd.DataFrame(data=new_genres_array, columns=anime_genres_columns)

# 把 genre 丢掉
anime_train_df = anime_train_df.drop(labels=['genre', 'index'],axis=1)
anime_train_df

,anime_id,rating
0,32281,9.37
1,5114,9.26
2,28977,9.25
3,9253,9.17
4,9969,9.16
...,...,...
11949,9316,4.15
11950,5543,4.28
11951,5621,4.88
11952,6133,4.98


In [38]:
# 把 type 提出来接在前面
# id_type_df = anime_train_df.drop(labels=['rating'],axis=1)
# id_type_df = id_type_df.assign(int)
# 把 rating 提出来接在最后 
# id_rating_df = anime_train_df.drop(labels=['type'],axis=1)

anime_train_df_new = pd.merge(how='left', left=anime_train_df, right=anime_train_genres_df)   
# anime_train_df_new = pd.merge(how='left', left=id_rating_df, right=anime_train_df_new)  

anime_train_df_new = anime_train_df_new.drop(labels=['anime_id'],axis=1)


In [39]:
anime_train_df_new.loc [anime_train_df_new['rating'] <= 6.0, 'rating'] = 0.0
anime_train_df_new.loc [anime_train_df_new['rating'] >= 6.0, 'rating'] = 1.0

anime_train_df_new.replace(np.nan, 0.0, inplace=True) 
anime_train_df_new.replace(np.inf, 0.0, inplace=True)
anime_train_df_new =anime_train_df_new.astype(int)

# indexNames = anime_train_df[ anime_train_df['rating'] == inf ].index
# # print(indexNames)
# # Delete these row indexes from dataFrame
# anime_train_df.drop(indexNames , inplace=True)
# anime_train_df = anime_train_df.reset_index()
# anime_train_df.episodes.astype(int)

# anime_train_df = anime_train_df.join(anime_train_genres_df)
anime_train_df_new


,rating,Action,Adventure,Cars,Comedy,Dementia,Demons,Drama,Ecchi,Fantasy,...,"Slice, of, Life",Space,Sports,"Super, Power",Supernatural,Thriller,Vampire,Yaoi,Yuri,nan
0,1,0,0,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
1,1,1,1,0,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
2,1,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,1,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11949,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11950,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11951,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11952,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [40]:
# 新的 data 会长成 id, type, genres[], rating
# anime_train_df_new = anime_train_df_new.drop(labels=['anime_id', 'type'],axis=1)

anime_train_df_new.to_csv('data/anime_genres_splitted.csv',index=False, header=None)
anime_train_df_new.info()
# anime_train_df_new.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11954 entries, 0 to 11953
Data columns (total 45 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   rating           11954 non-null  int32
 1   Action           11954 non-null  int32
 2   Adventure        11954 non-null  int32
 3   Cars             11954 non-null  int32
 4   Comedy           11954 non-null  int32
 5   Dementia         11954 non-null  int32
 6   Demons           11954 non-null  int32
 7   Drama            11954 non-null  int32
 8   Ecchi            11954 non-null  int32
 9   Fantasy          11954 non-null  int32
 10  Game             11954 non-null  int32
 11  Harem            11954 non-null  int32
 12  Hentai           11954 non-null  int32
 13  Historical       11954 non-null  int32
 14  Horror           11954 non-null  int32
 15  Josei            11954 non-null  int32
 16  Kids             11954 non-null  int32
 17  Magic            11954 non-null  int32
 18  Martia

In [41]:
# genre_map = {}
# for index in range(len(genre_set)):
#     genre_map[genre_set[index]] = index
# print(genre_map)


In [42]:

# f = open('test.txt', 'w')
# for genre in genre_set:
#     f.write(genre)
#     f.write('\n')
# f.close()

# test = pd.get_dummies(anime_train_df.genre)